In [1]:
# In colab please uncomment this to install Atari 
# Box2d is a 2D physics engine.
!pip install box2d-py

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# And for visualization on Colab install
!pip install pyglet
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import gym

/home/ana007652/.local/lib/python3.6/site-packages/gym/core.py:27: UserWarning: WARN: Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from IPython import display as ipythondisplay  ## Needed on colab

In [5]:
## Need to set a virtual Display on colab, else ipythondisplay would not work
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb': 'Xvfb'

## Physics Engine

In [7]:
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.9 MB 6.5 MB/s eta 0:00:01


In [8]:
env = gym.make('LunarLander-v2')
obs = env.reset() 
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)
#ipythondisplay.clear_output(wait=True)  #Colab

TypeError: render() got an unexpected keyword argument 'mode'

# Classic Control 

In [9]:
env = gym.make('CartPole-v0')
env.reset()
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)

/home/ana007652/.local/lib/python3.6/site-packages/gym/envs/registration.py:556: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "


TypeError: render() got an unexpected keyword argument 'mode'

## Atari Games

In [10]:
!wget http://www.atarimania.com/roms/Roms.rar
!mkdir /content/ROM/
!unrar e /content/Roms.rar /content/ROM/
!python -m atari_py.import_roms /content/ROM/

--2023-09-10 15:43:55--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19612325 (19M) [application/x-rar-compressed]
Saving to: ‘Roms.rar.2’

100%[======================================>] 19,612,325   773KB/s   in 25s    

2023-09-10 15:44:21 (760 KB/s) - ‘Roms.rar.2’ saved [19612325/19612325]

mkdir: cannot create directory ‘/content/ROM/’: No such file or directory
/bin/bash: unrar: command not found
/bin/python: No module named atari_py


In [11]:
env = gym.make('SpaceInvaders-v0')
env.reset()
img = env.render(mode='rgb_array')
env.close()
plt.imshow(img)

NameNotFound: Environment SpaceInvaders doesn't exist. 

In [12]:
from gym import envs

envall = envs.registry.all()
len(envall)

AttributeError: 'dict' object has no attribute 'all'

In [13]:
from tqdm import tqdm
List = []
for e in tqdm(envall):
    try:
        env = e.make()
        List.append([e.id, env.observation_space, env.action_space, env.reward_range])
        env.close() 
    except:
        continue     
    


NameError: name 'envall' is not defined

In [ ]:
df = pd.DataFrame(List, columns= ['Environment', 'Observation Space', 'Action Space', 'Reward'])
df.sample(20)

In [14]:
env = gym.make('MountainCar-v0')
print(f"The Observation space is        {env.observation_space}" )

print(f"Upper Bound for Env Observation {env.observation_space.high}")
print(f"Lower Bound for Env Observation {env.observation_space.low}")
print(f"Action Space                    {env.action_space}")

env.seed(0)
obs = env.reset()
print(f"The initial observation is      {obs}")
# # Take a random actionget the new observation space
new_obs, reward, done, info = env.step(env.action_space.sample())
print(f"The new observation is          {new_obs}")
env.close()

The Observation space is        Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Upper Bound for Env Observation [0.6  0.07]
Lower Bound for Env Observation [-1.2  -0.07]
Action Space                    Discrete(3)


AttributeError: 'MountainCarEnv' object has no attribute 'seed'

# Working with Wrappers

In [15]:
class ClippedRewards(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)
        self.reward_range = (-10,10)

    def reward(self, reward):
        """Clip to {+10, 0, -10} by its sign."""
        return reward if reward >= -10 and reward <= 10 else 10 * np.sign(reward)

In [16]:
env = gym.make("CartPole-v0")
print("Original Rewards:", env.reward_range)
env.close()
env = ClippedRewards(gym.make("CartPole-v0"))
print(f'Clipped reward range: {env.reward_range}')
env.close()


Original Rewards: (-inf, inf)
Clipped reward range: (-10, 10)


# Working with Wrappers

In [17]:
from collections import deque
from gym import spaces
import numpy as np

class ConcatObservations(gym.Wrapper):
    def __init__(self, env, n):
        gym.Wrapper.__init__(self, env)
        shape = env.observation_space.shape
        self.n = n
        self.frames = deque([], maxlen=n)
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((n,) + shape), dtype=env.observation_space.dtype)
    def reset(self):
        obs = self.env.reset()
        for _ in range(self.n):
            self.frames.append(obs)
        return self._get_obs()

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.frames.append(obs)
        return self._get_obs(), reward, done, info

    def _get_obs(self):
        return np.array(self.frames)

In [18]:
env = gym.make("BreakoutNoFrameskip-v4")
print(f"The original observation space is  {env.observation_space}")
env = ConcatObservations(env, 4)
print(f"The new observation space is  {env.observation_space}")


NameNotFound: Environment BreakoutNoFrameskip doesn't exist. 

In [ ]:
env.reset()
obs, _, _, _  = env.step(env.action_space.sample())
print(f"Observation after taking a random step is  {obs.shape}")
env.close()

# Monitors for saving videos

In [ ]:
import gym
from gym.wrappers import Monitor
#env = gym.make('CartPole-v0')
env = Monitor(gym.make('CartPole-v0'), 'recording', force=True)
state = env.reset()
done = False

while not done:
    action = env.action_space.sample()
    state_next, reward, done, info = env.step(action)
    
env.close()


In [ ]:
# On Colab
from IPython.display import HTML
from base64 import b64encode
mp4 = open('recording/openaigym.video.3.59.video000000.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


## Visualize Animation

In [ ]:
import gym
import matplotlib.pyplot as plt
import matplotlib.animation as animation

env = gym.make('Pong-v0')
obs = env.reset()
frames = [] # array to store state space at each step 
for _ in range(300):
    frames.append(env.render(mode='rgb_array')) 
    obs,reward,done, _ = env.step(env.action_space.sample()) 
    if done:
        break


In [ ]:
patch = plt.imshow(frames[0])
plt.axis('off')
def animate(i):
    patch.set_data(frames[i])
anim = animation.FuncAnimation(plt.gcf(), animate, \
                               frames=len(frames), interval=100)
#anim.save('random_agent.gif', writer='imagemagick')

![](https://github.com/Educat8n/Reinforcement-Learning-for-Game-Playing-and-More/blob/main/images/random_agent.gif?raw=1)

In [ ]:
# For Colab
from IPython.display import HTML
HTML(anim.to_html5_video())